**Scene:**

Its 3pm and your supervisor has just handed you their old Fortran77 code that they swear "solved this problem in the 80s" and "is really smartly written and easy to follow". After the meeting you take one look at the code and realize it doesn't. And it's not. 

But, you said you'd have results by the group meeting the following day, so lets see if we can understand what it is doing, fix it up, and get some results!

$_{\rm Also,\ this\ code\ has\ somehow\ already\ been\ converted\ into\ python\ 3.\ who\ knows}$






**In this question we will try to develop some smart coding practices, and try to improve upon some poorly written code.**

You are provided with the following code in this workbook, 
which is both memory and computationally inefficient (and poorly commented)

Your task is to speed the code up as much as possible, and to make it as memory efficient as possible.
Also, you should comment "important" lines with what operations they seem to be performing

Hints:

**1.) Try to get rid of every single for loop (vectorize everything you can)**

**2.) Get rid of duplicate memory allocation**

> e.g. 

> array1 = np.zeros((10,10))

> array2 = array1 * 2.0

> array3 = array2**2 

> Instead just use array1 = array1 * 2.0 ; array1 = array1**2, etc.


**Marks will be based on:**

1.) run time of your improved code compared to the original (0 for loops are needed)

2.) memory usage of your code compared to the original

3.) Documentation. Fix missing comments (hint: look for lines that start with "###"), and feel free to add additional ones if you feel they are necessary

**Lets first test some good coding practices here:**

1.) **Never use for loops unless you have to - super slow! Vectorize everything**

e.g.:

In [6]:
# Make random data arrays

import numpy as np
import time

N = 10000000
A = np.arange(N)
B = np.random.uniform(size=N)

print("data created")

# test time to use for loop vs vectorized version

# Bad Way
start = time.time()
total = 0
for i in range(N):
    total += A[i] * B[i]
end = time.time()
print("For loop: total = %d in time = %d seconds"%(total, end-start))

# Good Way
start = time.time()
total = np.sum(A*B)
end = time.time()
print("Vectorized: total = %d in time = %d seconds"%(total, end-start))


data created
For loop: total = 25002157026354 in time = 30 seconds
Vectorized: total = 25002157026354 in time = 0 seconds


2.) **Don't convert lists to arrays/array to lists unless necessary for some reason**

e.g.:


In [7]:
N = 10000000

# Bad Way
start = time.time()
A = []
for i in range(N):
    A_i = np.random.uniform()
    A.append(A_i)

A = np.array(A)
end = time.time()
print("List/For loop: total = %d in time = %d seconds"%(total, end-start))


# Good Way
start = time.time()
A = np.random.uniform(size=N)
end = time.time()
print("Array Vectorized: total = %d in time = %d seconds"%(total, end-start))


List/For loop: total = 25002157026354 in time = 10 seconds
Array Vectorized: total = 25002157026354 in time = 0 seconds


3.) **Don't duplicate arrays unless you actually need the multiple versions**

e.g.:

In [8]:
A = np.random.uniform(size=(100,100))
B = A*2
C = B**2

# you've now tripled the memory usage to calculate C from A. Unless you actually need A and B later, don't do this. 
# Just do:
A = np.random.uniform(size=(100,100))
A = A*2
A = A**2

# or 

A = np.random.uniform(size=(100,100))
A *= 2
A = A**2

# or whatever

**Code Outline**
The code you will be working with is a mockup of a three dimensional "large scale structure survey", or a "mock" observation. Using this code we want to populate our field of view with the flux from a number (Nhalo) of dark matter halos. 

Each of these halos will be given:

1.) a randomly drawn position in (ra [deg], dec [deg], distance [Mpc (comoving)])

2.) a randomly drawn Mass in [Msun], drawn from an analytical Halo Mass Function (HMF) 


For each of these halos you will calculate a Luminosity, based on the given Mass-to-Luminosity relation mass_to_luminosity(mass). 

You will then bin the luminosity of these halos into a three dimensional map/array of size (npix_x, npix_y, npix_z) which represents (ra, dec, distance - aka a three dimensional data cube, where each voxel (3D pixel) represents the luminosity coming from that region of space.)

You will then perform a simple "analysis" of this map 

**first, a bit on what a "dark matter halo mass function" is:**

The Halo Mass function describes the number of dark matter halos of a given mass M

For this example we will use a simplified version of the form HMF = M * np.exp(-M/Mstar), where M is the mass of a halo

How do we randomly draw halo mass values according to a distribution? **This is a very common type of excercise for reserch/job interviews.**

Probability distrution function PDF: 

$f(x)= A x exp(-x/b)$

where $A = \int_0^{inf} f(x) dx$

and b is given

Cumlative distribution function CDF: 

$ F(x) = \int_0^x f(x)dx $

so

$F(x) = Ab(b-(x+b)exp(-x/b))$

Cannot invert this analytically, so must do numerically.

With the inverted CDF, CDFinv, we can simply draw a uniform random number in the range (0,1), plug this into the CDFinv, and it will return samples drawn from any PDF that we desire!


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from scipy import interpolate
import time

tstart = time.time()

# Halo Parameters
Nhalo = 100000000 # set to 100000000 for final run (Note large numbers will probably not even work with the original poorly written code)

# Map parameters
npix = 1000 # set to 1000 for final run

fov_x = 10 # in degrees
fov_y = 10 # in degrees
chi_min = 0. # minimum distance
chi_max = 3.e3 # maximum distance

###v
dx_pix = fov_x/npix
dy_pix = fov_y/npix
dz_pix = (chi_max - chi_min)/npix

ra = np.random.uniform(-fov_x/2, fov_x/2, Nhalo)
dec = np.random.uniform(-fov_y/2, fov_y/2, Nhalo)
chi = np.random.uniform(chi_min, chi_max, Nhalo)
### 

# Set max and Min mass so integral converges
Mmin  = 0.
Mmax  = 1.e16
Mstar = 1.e13

###??
hmf_func = lambda M: M*np.exp(-M/Mstar)
norm_pdf, err = integrate.quad(hmf_func, Mmin, Mmax)

hmf_pdf = lambda x: 1./norm_pdf * x*np.exp(-x/Mstar)

###??
hmf_cdf = lambda x: 1./norm_pdf * Mstar * (Mstar - (x + Mstar)*np.exp(-x/Mstar))

# Make array of masses
M = np.linspace(Mmin, Mmax, 10000)

# Get CDF value at each M
CDF_of_M = hmf_cdf(M)

### ??
CDF_of_M_inv = interpolate.interp1d(CDF_of_M, M)

# Does drawing from this give the correct mass function?? Check 

###v
mass = CDF_of_M_inv(np.random.uniform(size=Nhalo))

###v
mass_to_luminosity = lambda x: (x/Mstar)**2.4 

Lum = mass_to_luminosity(mass)
Flux = Lum/4/np.pi/chi**2

###v?
map_1 = np.zeros((npix,npix,npix))
i = ((ra +fov_x/2) // dx_pix).astype(int)
j = ((dec+fov_y/2) // dy_pix).astype(int)
k = (chi // dz_pix).astype(int)

for h in range(Nhalo):
    map_1[i[h], j[h], k[h]] += Flux[h]
                

###v
    
chi_cut = 500
map_sum = np.zeros((npix,npix))

#k would range from 0 to npix-1 but we pick out k's that dont make the cut
k = np.arange(0,int(chi_cut/dz_pix)+1, dtype=int) # values of k's that DON'T make the cut
map_sum = np.copy(map_1) # copy map_1 or else we will mutate map_1
map_sum[:,:,k] = 0 # all values before axis 2 index k will be zero 
map_sum = np.sum(map_sum, axis=2) # sum all values in axis 2
npix_include = npix-len(k) # number of k's that DO make the cut

map_mean = map_sum/npix_include
hist, bin_edges = np.histogram(map_mean)

tend = time.time()

print("total runtime = %d seconds" % (tend-tstart))                

KeyboardInterrupt: 

In [2]:
# Analyze the map "map_mean"

# What does this map_mean represent? Plot it. 

# Take the histogram of the pixel intensities. Plot that

# find the indices of the 10 brightest pixels. Print those out 